<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from tqdm import tqdm

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score


In [2]:
pd.set_option('display.max_rows', None)  # Отображать все строки
pd.set_option('display.max_columns', None)  # Отображать все столбцы

## Подготовка

Загрузим данные.

In [3]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv') #чтение файла /datasets/toxic_comments.csv
display (toxic_comments.head(20)) #вывод первых строк
print (toxic_comments.info()) #вывод общей информации о DataFrame

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB
None


Первый вывод - в таблице нет пустых значений.
Проверим наличие явных дубликатов.

In [4]:
print('Количество дубликатов:', toxic_comments.duplicated(subset=['text', 'toxic']).sum())

Количество дубликатов: 0


В курсе приводятся примеры лемматизации для русского языка. В некоторых источниках указано, что если использовать для английского текста TfidfVectorizer, то токенизация, стоп-слова и нормализация уже встроены.

In [5]:
# Разделим данные на признаки и целевой признак
texts = toxic_comments['text']
target = toxic_comments['toxic']

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(texts, target, test_size=0.2, random_state=42)

# Функция очистки текста
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Удалим все символы, кроме латинских букв
    return text

# Очистим текст
X_train_cleaned = X_train.apply(clean_text)
X_test_cleaned = X_test.apply(clean_text)


# Лемматизация
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):
    words = word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words])

tqdm.pandas()

X_train_lemmatized = X_train_cleaned.progress_apply(lemmatize_text)
X_test_lemmatized = X_test_cleaned.progress_apply(lemmatize_text)

# Векторизация
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_vectorized = vectorizer.fit_transform(X_train_lemmatized)
X_test_vectorized = vectorizer.transform(X_test_lemmatized)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
100%|██████████| 31859/31859 [03:42<00:00, 143.04it/s]


## Обучение

In [6]:
# Список моделей и гиперпараметров
models_and_params = {
    "Logistic Regression": (
        LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced', solver='liblinear'),
        {'C': [5, 10, 15], 'penalty': ['l1', 'l2']}
    ),
    "Linear SVC": (
        LinearSVC(random_state=42, max_iter=5000, class_weight='balanced'),
        {'C': [5, 10, 15]}
    )
}

cv_results = {}

print("Поиск лучших гиперпараметров и сравнение моделей по F1:")

for name, (model, param_grid) in models_and_params.items():
    grid = GridSearchCV(model, param_grid, scoring='f1', cv=5, n_jobs=-1)
    grid.fit(X_train_vectorized, y_train)
    
    best_f1 = grid.best_score_
    best_params = grid.best_params_
    cv_results[name] = (best_f1, best_params)
    
    print(f"{name}:")
    print(f"  Лучшая F1: {round(best_f1, 3)}")
    print(f"  Лучшие параметры: {best_params}\n")

# Определяем лучшую модель
best_model_name = max(cv_results, key=lambda name: cv_results[name][0])
best_f1, best_params = cv_results[best_model_name]

print(f"Лучшая модель: {best_model_name}")

Поиск лучших гиперпараметров и сравнение моделей по F1:
Logistic Regression:
  Лучшая F1: 0.758
  Лучшие параметры: {'C': 5, 'penalty': 'l2'}

Linear SVC:
  Лучшая F1: 0.736
  Лучшие параметры: {'C': 5}

Лучшая модель: Logistic Regression


In [7]:
# Финальное тестирование лучшей модели
best_model_class, param_grid = models_and_params[best_model_name]
best_params = cv_results[best_model_name][1]

# Обучаем модель с лучшими параметрами на всём тренировочном наборе
final_model = best_model_class.set_params(**best_params)
final_model.fit(X_train_vectorized, y_train)

# Предсказания и F1 на тестовой выборке
y_pred = final_model.predict(X_test_vectorized)
f1 = f1_score(y_test, y_pred)
print(f"\nF1-мера на тестовой выборке для {best_model_name}: {round(f1, 3)}")


F1-мера на тестовой выборке для Logistic Regression: 0.76


Прекрасно! По ТЗ необходимо было достичь показателя F1 не менее 0.75.
F1-мера на тестовой выборке для Logistic Regression: 0.76

## Выводы

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. 
В проекте требовалось создать инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

В рамках проекта была выполнена предобработка данных, обучены модели с целью классифицировать комментарии на позитивные и негативные. 

По ТЗ необходимо было достичь показателя F1 не менее 0.75.
F1-мера на тестовой выборке для Logistic Regression: 0.76